In [2]:
import os

# import langchain
# from langchain.prompts import PromptTemplate
# from langchain.schema import StrOutputParser,chat_history
# from langchain.llms import OpenAI

from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI

# from langchain.memory import ConversationBufferMemory


from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain_community.document_loaders import PyPDFLoader, PyPDFium2Loader

# from creditcard.utils import get_current_time


# CHOOSE GPT VERSION (3.5 or 4)s
GPT = 3.5

if GPT == 3.5:
    print("USING GPT 3.5 Turbo")
    OPENAI_API_KEY = "95058a9e99794e4689d179dd726e7eec"
    OPENAI_DEPLOYMENT_NAME = "vassar-turbo35"
    OPENAI_API_BASE = "https://vassar-openai.openai.azure.com/"
    MODEL_NAME = "gpt-35-turbo"
    OPENAI_API_TYPE = "azure"
    OPENAI_API_VERSION = "2023-07-01-preview"

elif GPT == 3.55:
    # GPT 3.5 - instruct - turbo
    print("USING GPT 3.5 Instruct TURBO")
    OPENAI_API_KEY = "95058a9e99794e4689d179dd726e7eec"
    OPENAI_DEPLOYMENT_NAME = "vassar-instruct-turbo-35"
    OPENAI_API_BASE = "https://vassar-openai.openai.azure.com/"
    MODEL_NAME = "gpt-35-turbo-instruct"
    OPENAI_API_TYPE = "azure"
    OPENAI_API_VERSION = "2023-07-01-preview"


elif GPT == 4:
    print("USING GPT 4")
    OPENAI_API_KEY = "2752b82e2e3d41d59df518f56c7f9b3a"
    OPENAI_DEPLOYMENT_NAME = "vassar-gpt-4"
    OPENAI_API_BASE = "https://vassar-openai-canada.openai.azure.com/"
    MODEL_NAME = "gpt-4"
    OPENAI_API_TYPE = "azure"
    OPENAI_API_VERSION = "2023-07-01-preview"

OPENAI_EMBEDDING_MODEL_API_KEY = "95058a9e99794e4689d179dd726e7eec"
OPENAI_EMBEDDING_MODEL_DEPLOYMENT_NAME = "vassar-text-ada002"
OPENAI_EMBEDDING_MODEL_API_BASE = "https://vassar-openai.openai.azure.com/"
OPENAI_EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

os.environ["OPENAI_API_TYPE"] = OPENAI_API_TYPE
os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION
os.environ["OPENAI_API_BASE"] = OPENAI_API_BASE
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# LLM
gpt4_llm = AzureChatOpenAI(
    deployment_name=OPENAI_DEPLOYMENT_NAME,
    model_name=MODEL_NAME,
    temperature=0,
)


# gpt3_llm = AzureOpenAI(
#             deployment_name=OPENAI_DEPLOYMENT_NAME,
#             model_name=MODEL_NAME,
#             openai_api_base=OPENAI_API_BASE,
#             openai_api_type=OPENAI_API_TYPE,
#             openai_api_key=OPENAI_API_KEY,
#             openai_api_version=OPENAI_API_VERSION,
#             temperature = 0.0
#         )

USING GPT 3.5 Turbo


/home/premkumar/ccp-venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
/home/premkumar/ccp-venv/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://vassar-openai.openai.azure.com/ to https://vassar-openai.openai.azure.com/openai.
  warnings.warn(
/home/premkumar/ccp-venv/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:174: Us

In [3]:
from pprint import pprint
import json

In [6]:
class llm_chain:
    def __init__(self, prompt_template):
        prompt = ChatPromptTemplate(
            messages=[
                SystemMessagePromptTemplate.from_template(prompt_template),
                # MessagesPlaceholder(variable_name="chat_history"),
                # HumanMessagePromptTemplate.from_template("{question}"),
            ],
            input_variables=["email_content"],
        )

        # memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, input_key="question")
        self.chain = LLMChain(llm=gpt4_llm, prompt=prompt, verbose=True)

    def run(self, inp):
        # response = self.chain.run({question: question, name: name})
        # time = get_current_time()
        # print("hitting llm ",time)
        response = self.chain.run(inp)

        # time = get_current_time()
        # print("done ",time)

        return response

In [7]:
INTENT_CLASSIFICATION_PROMPT = """
You are an advanced email service AI system tailored for a company specializing in purchase orders, \
the system is designed to intelligently classify the intent of incoming emails related to purchase orders, discern the emotional tone conveyed within the message and \
Purchase Order ID/number if mentioned.

Intent Classification:
- Identify if the email pertains to a new purchase order (PO): "NEW_PO."
- If it is an inquiry about the status of an existing purchase order, label it as "PO_STATUS."
- If the intent cannot be confidently identified, categorize it as "UNIDENTIFIED."
- Ensure accurate intent classification based on a comprehensive analysis of both the subject and body of the email.

Emotional Tone Detection:
- Classify the emotional temperature of the message, indicating whether the customer is expressing expedite, anger, satisfaction, or a neutral sentiment.
- Recognize emotional states such as expedite, angry, satisfied, or neutral to provide a nuanced understanding.

Email:
{email_content}

Output Format:
```json
{{intent: [Intent Classified], tone: [Tone Detected], po_id:[Purchase Order ID/number]}}
```
"""

In [8]:
INTENT_CLASSIFICATION_PROMPT = """You are an advanced email service AI system designed to classify the intent of incoming emails and discern the emotional tone conveyed within the message. Your task is to intelligently categorize the emails into different intents and sub-intents based on their content.

Intents:
1. PURCHASE_ORDER:
   - Description: Managing purchase orders, including creation, status checks, and cancellation.
   - Sub-Intents:
     - NEW_PO: Creating a new purchase order.
     - CHECK_PO_STATUS: Checking the status of a purchase order.
     - PO_CANCELLATION: Requesting cancellation of a purchase order.

2. SHIPMENT_STATUS:
   - Description: Checking the status of a shipment.
   - Sub-Intents:
     - CHECK_SHIPMENT_STATUS: Checking the status of a shipment.

3. WARRANTY:
   - Description: Handling warranty-related inquiries and claims.
   - Sub-Intents:
     - INITIATE_WARRANTY_CLAIM: Initiating a warranty claim for faulty equipment.

4. SERVICE_REQUESTS:
   - Description: Requesting various services, maintenance, or troubleshooting tasks.
   - Sub-Intents:
     - ROUTINE_MAINTENANCE: Scheduled maintenance tasks to ensure functionality.
     - TROUBLESHOOTING: Diagnosing and resolving technical issues.
     - INSTALLATION: Installing new items or replacing existing ones.
     - UPGRADES_MODIFICATIONS: Upgrading or modifying items to enhance performance.

5. RESOLVED:
   - Description: Classifying emails that indicate the resolution of a previous issue or request.


Emotional Tone Detection:

- Classify the emotional tone of the message, indicating whether the customer is expressing HAPPY, ANXIOUS, UNHAPPY, or SATISFIED.

- Please analyze the emotional tone of the provided message and determine the predominant sentiment expressed by the customer. Choose the most appropriate emotional state from the following options:

HAPPY: The customer expresses positivity, joy, or contentment.
ANXIOUS: The customer expresses worry, nervousness, or apprehension.
UNHAPPY: The customer expresses negativity, dissatisfaction, or sadness.
SATISFIED: The customer expresses contentment, fulfillment, or approval.

Select the emotional state that best reflects the overall sentiment conveyed in the message. If the emotional tone is mixed or unclear, choose the category that aligns most closely with the prevailing emotion.

Email:
{email_content}

Output Format:
```json
{{
  "intent": "[Main Intent Classified]",
  "sub_intent": "[Sub-Intent Classified, if applicable]",
  "tone": "[Emotional Tone Detected]"
}}
```
"""

In [9]:
ORDER_DETAILS_EXTRACTION = """
Extract the following details from the provided Purchase Order Content:

1. Purchase Order Number/ID
2. Product details including:
   - Product Name
   - Quantity
   - Price per Unit
   - Total Net Value
3. Total Net Order Value (Excluding Tax)
4. Contact Person details including:
   - Name
   - Telephone/Phone Number
   - Email
   - Address
5. Estimated Time of Delivery

Purchase Order Content:

{po_content}

Output Format:
Format everthing into JSON

```json
{{po_id:[Purchase Order Number],
contact_person_details:{{name:[Name],phone_number:[Telephone/Phone Number], email:[Email], address: [Address]}},
products:[{{name:[Product Name],quantity:[Quantity], price_per_unit:[Price Per Unit (without currency)], net_value:[Total Net Value]}}],
eta:[Estimated Time of Delivery YYYY-MM-DD]
}}
```

"""

In [40]:
CONTACT_DETAILS_EXTRACTION = """
Your are trained on understanding each detail in email content. 
Your task is to extract the following details:
1. Customer Details:
  - Customer Name
2. Geography of the customer
  - state, country, address
Email Content:{email_content}
Output Format:
Format everthing into JSON
```json
{{
customer_name:[Customer Name]
locations:[state:[State],country:[Country],Address:[Address]]
}}
```


Remember: If you are ambigous or unsure at idenitifying any detail, please output with empty string
"""

In [46]:
email_content = """
Subject: TP6EBS13 - Oracle Sr # :13916210 Customer : Walgreen National
Corp SR Group : CHECKPOINT_SYSTEMS Status: Open
Hello Team,
Oracle SR : &quot;13916210&quot; created/updated on : 11-MAR-2024 09:29:35 AM EST
Customer Name : Walgreen National Corp

Customer Account (Torrico) : 6131853
Assigned to : CHECKPOINT_SYSTEMS
Task : 14756577
SR Severity : Sev 3 WGN
SR Problem Summary : EAS - No Alarms At Entrance, Exit Or Interior Gates
Item : 0368-36W
Item Description : Walgreens - Checkpoint QS2000 Dual Antenna Detection System
Target Arrival Date/Time : 13-MAR-2024 13:00:00 EST
Resolve By Date/Time : 13-MAR-2024 15:00:00 EST
Contact Name : Bethany Jackson Walgreens
Contact Number : 615 848 0968
Address :2401 Old Fort Pkwy, , Walgreens , 06277
City : Murfreesboro
State/Province : TN
Postal Code : 37128-4162
Country : US
Regards
Oracle Ebiz

"""

In [47]:
chain = llm_chain(INTENT_CLASSIFICATION_PROMPT)

result = chain.run({"email_content": email_content})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are an advanced email service AI system designed to classify the intent of incoming emails and discern the emotional tone conveyed within the message. Your task is to intelligently categorize the emails into different intents and sub-intents based on their content.

Intents:
1. PURCHASE_ORDER:
   - Description: Managing purchase orders, including creation, status checks, and cancellation.
   - Sub-Intents:
     - NEW_PO: Creating a new purchase order.
     - CHECK_PO_STATUS: Checking the status of a purchase order.
     - PO_CANCELLATION: Requesting cancellation of a purchase order.

2. SHIPMENT_STATUS:
   - Description: Checking the status of a shipment.
   - Sub-Intents:
     - CHECK_SHIPMENT_STATUS: Checking the status of a shipment.

3. WARRANTY:
   - Description: Handling warranty-related inquiries and claims.
   - Sub-Intents:
     - INITIATE_WARRANTY_CLAIM: Initiating a warranty claim for faulty equipment.



In [48]:
print(result)

{
  "intent": "SERVICE_REQUESTS",
  "sub_intent": "TROUBLESHOOTING",
  "tone": "SATISFIED"
}


In [49]:
chain = llm_chain(CONTACT_DETAILS_EXTRACTION)

result = chain.run({"email_content": email_content})



> Entering new LLMChain chain...
Prompt after formatting:
System: 
Your are trained on understanding each detail in email content. 
Your task is to extract the following details:
1. Customer Details:
  - Customer Name
2. Geography of the customer
  - state, country, address
Email Content:
Subject: TP6EBS13 - Oracle Sr # :13916210 Customer : Walgreen National
Corp SR Group : CHECKPOINT_SYSTEMS Status: Open
Hello Team,
Oracle SR : &quot;13916210&quot; created/updated on : 11-MAR-2024 09:29:35 AM EST
Customer Name : Walgreen National Corp

Customer Account (Torrico) : 6131853
Assigned to : CHECKPOINT_SYSTEMS
Task : 14756577
SR Severity : Sev 3 WGN
SR Problem Summary : EAS - No Alarms At Entrance, Exit Or Interior Gates
Item : 0368-36W
Item Description : Walgreens - Checkpoint QS2000 Dual Antenna Detection System
Target Arrival Date/Time : 13-MAR-2024 13:00:00 EST
Resolve By Date/Time : 13-MAR-2024 15:00:00 EST
Contact Name : Bethany Jackson Walgreens
Contact Number : 615 848 0968
Addres

In [50]:
pprint(json.loads(result))

{'customer_name': 'Walgreen National Corp',
 'locations': {'address': '2401 Old Fort Pkwy, Walgreens, 06277',
               'country': 'US',
               'state': 'TN'}}


In [34]:
PO_CONFIRMATION_SUMMARY_AND_RESPONSE_PROMPT = """
You are an advanced email service AI system tailored for a company specializing in purchase orders.
Your task is to generate a concise 2-line SUMMARY for the customer service team to quickly understand the email content. 
Moreover, craft a detailed EMAIL RESPONSE confirming the purchase order. 
This response should include order details, express gratitude, and assure the customer of efficient processing and timely delivery.

Customer Name: {customer_details}
Purchase Order Number: {po_id}
Order Date: {order_date}
Delivery Address: {contact_person_details}
Products : {products}
Expected Delivery Date: {eta}
Company Name : {vendor_details}

Output Format:
```json
{{summary: [SUMMARY], response:[EMAIL_RESPONSE]}}
```

Ensure the generated SUMMARY is concise yet informative, and the EMAIL RESPONSE is professionally structured, confirming the purchase order with the provided details.
"""

In [35]:
order_confirmation_chain = llm_chain(
    prompt_template=PO_CONFIRMATION_SUMMARY_AND_RESPONSE_PROMPT,
)

In [11]:
order_data = {
    "customer_details": {
        "customer_name": "XYZ Corporation",
        "customer_address": "789 Business Street, Metropolis",
        "billing_address": "789 Finance Avenue, Metropolis",
    },
    # 'order_details':{
    "po_id": "PO2024004",
    "order_date": "January 20, 2024",
    "contact_person_details": {
        "contact_person": "John Smith",
        "phone": "555-1234",
        "email": "john.smith@xyzcorp.com",
    },
    "products": [
        {"item": "Office Chairs", "quantity": 50, "unit_price": 150, "total": 7500},
        {"item": "Desks", "quantity": 20, "unit_price": 300, "total": 6000},
        {"item": "Conference Tables", "quantity": 5, "unit_price": 500, "total": 2500},
    ],
    "eta": "February 10, 2024",
    # },
    "vendor_details": {
        "company_name": "ABC Furniture Inc.",
        "company_address": "123 Supplier Avenue, Cityville",
    },
}

In [12]:
result = order_confirmation_chain.run(order_data)
print(result)



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are an advanced email service AI system tailored for a company specializing in purchase orders.
Your task is to generate a concise 2-line SUMMARY for the customer service team to quickly understand the email content. 
Moreover, craft a detailed EMAIL RESPONSE confirming the purchase order. 
This response should include order details, express gratitude, and assure the customer of efficient processing and timely delivery.

Customer Name: {'customer_name': 'XYZ Corporation', 'customer_address': '789 Business Street, Metropolis', 'billing_address': '789 Finance Avenue, Metropolis'}
Purchase Order Number: PO2024004
Order Date: January 20, 2024
Delivery Address: {'contact_person': 'John Smith', 'phone': '555-1234', 'email': 'john.smith@xyzcorp.com'}
Products : [{'item': 'Office Chairs', 'quantity': 50, 'unit_price': 150, 'total': 7500}, {'item': 'Desks', 'quantity': 20, 'unit_price': 300, 'total': 6000}, {'item': 'Confe

In [13]:
import json

json_data = json.loads(result)
# print(f"Summary:\n{json_data['summary']}")

print(f"Email:\n{json_data['response']}")

Email:
Dear XYZ Corporation,

Thank you for your purchase order PO2024004 for office furniture. We have received your order for 50 office chairs, 20 desks, and 5 conference tables. The total amount is $16,000. We appreciate your business and assure you that your order will be processed efficiently and delivered to the provided address by February 10, 2024. If you have any further questions or concerns, please feel free to contact us. 

Best regards,
ABC Furniture Inc.


In [95]:
PO_ISSUE_SUMMARY_AND_RESPONSE_PROMPT = """
You are an advanced email service AI system tailored for a company specializing in purchase orders given by customer.
Specifically focus is on identifying and addressing issues with product details. 
Your objective is to analyze provided information and generate a short sentence SUMMARY for the customer service team, offering a quick understanding of the key issues. 
Additionally, craft a comprehensive EMAIL RESPONSE that professionally communicates identified issues in the purchase order to relevant stakeholders, urging them to reply with a revised purchase order.

Company Name : {vendor_details}
Customer Details: {customer_details}

Input:
- Purchase Order Number: {po_id}
- A list of products with issues in the purchase order: {product_issues_list}

Task:

1. Generate Summary:
   - The AI system should analyze the list of products with issues and producing a short summary that effectively communicates the key problems with the product details in the purchase order.
   - Ensure that the summary is clear, informative.

2. Suggested Email Response:
   - Suggest a draft email response that professionally communicates the identified issues to the customer. 
   - This response should address each problem in detailed one-after-one, and, if necessary, request additional information or action.
   - Encourage the customer to respond with a revised purchase order.

Output Format:
```json
{{summary: [SUMMARY], response:[EMAIL_RESPONSE]}}
```
"""

In [96]:
po_rectification_chain = llm_chain(
    prompt_template=PO_ISSUE_SUMMARY_AND_RESPONSE_PROMPT,
    # input_variables=['customer_details','po_id','product_issues_list', 'vendor_details']
)

In [97]:
dummy_data = {
    "customer_details": {"name": "John Doe"},
    "po_id": "PO2024004",
    "product_issues_list": [
        "Product: Laptop, Requested product is not available \n",
        "Product: Desk, Price mentioned in the Purchase Order (PO) is mismatched with original price. "
        "Original Price: $300, Price Mentioned: $250.\n",
        "Product: Chair, The quantity specified in the PO doesn't match the available box size in inventory. "
        "Recommend with available box size: 10, instead of requested quantity: 8",
    ],
    "vendor_details": {
        "company_name": "ABC Electronics",
        "company_address": "456 Tech Avenue, Techcity",
    },
}

In [98]:
result = po_rectification_chain.run(dummy_data)



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are an advanced email service AI system tailored for a company specializing in purchase orders given by customer.
Specifically focus is on identifying and addressing issues with product details. 
Your objective is to analyze provided information and generate a short sentence SUMMARY for the customer service team, offering a quick understanding of the key issues. 
Additionally, craft a comprehensive EMAIL RESPONSE that professionally communicates identified issues in the purchase order to relevant stakeholders, urging them to reply with a revised purchase order.

Company Name : {'company_name': 'ABC Electronics', 'company_address': '456 Tech Avenue, Techcity'}
Customer Details: {'name': 'John Doe'}

Input:
- Purchase Order Number: PO2024004
- A list of products with issues in the purchase order: ['Product: Laptop, Requested product is not available \n', 'Product: Desk, Price mentioned in the Purchase Order (PO) is 

In [99]:
import json

# print(result)
json_data = json.loads(result)
print(json_data["response"], json_data["summary"])

Dear John Doe,

Thank you for your recent purchase order (PO2024004) with ABC Electronics. We have reviewed your order and identified the following issues with the product details:

1. Laptop: The requested product is not available in our inventory. We apologize for the inconvenience caused. Please let us know if you would like to choose an alternative product or if you would prefer a refund.

2. Desk: The price mentioned in the Purchase Order (PO) does not match the original price. The original price for the desk is $300, but the price mentioned in the PO is $250. We kindly request you to confirm the correct price for the desk.

3. Chair: The quantity specified in the PO does not match the available box size in our inventory. We recommend a quantity of 10, instead of the requested quantity of 8. Please let us know if you would like to proceed with the recommended quantity or if you have any other preferences.

We kindly request you to respond to this email with a revised purchase orde

In [6]:
!pip install pypdf


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [13]:
loader = PyPDFium2Loader(
    file_path="/home/premkumar/GenAI/emailServiceBot/email_service_backend/attachments/customer2_2024_01_18/PDF0225 P100000127638_1 New Purchase Order Printout - Coast to Coast.pdf"
)

pages = loader.load_and_split()

print(f"No. of pages: {len(pages)}")

for i in range(len(pages)):
    print(f"Page {i+1}: {pages[i].page_content}\n")
    print("-" * 10, "\n")

No. of pages: 2
Page 1: Terms of deliv.: DDP DELIVERY DUTY PAID
Terms of payt.: 45 DAYS FROM INV DATE Currency USD ____________________________________________________________________________________
Item Material Description
 Order qty. Unit Price per unit Net value
Drawing Rev Level
____________________________________________________________________________________
00001 Signing of the Agreement
 1 each 39,227.00 39,227.00
Deliv. date Day 10/26/2022 
DB Design and Design Document
00002 Development of Mockups
 1 each 39,227.00 39,227.00
Deliv. date Day 04/17/2023 
00003 User Management
 1 each 39,227.00 39,227.00
Deliv. date Day 04/17/2023 
Could based configuration of mobile inTake forms, Mobile application for inTake forms
00004 Alert Management
 1 each 39,227.00 39,227.00
Deliv. date Day 04/17/2023 
KPI analytics & Reports, Automated Report Generation
00005 Cyber security audit, go live of the Sol
 1 each 39,227.00 39,227.00
Deliv. date Day 04/17/2023 
Vendor Address : 
VASSAR LAB

In [1]:
import os

print(
    os.listdir(
        "/home/premkumar/GenAI/emailServiceBot/email_service_backend/attachments/customer2_2024_01_18"
    )
)

['PDF0225 P100000127638_1 New Purchase Order Printout - Coast to Coast.pdf', 'ExtractedFolder.zip', 'ExtractedFolder']


In [ ]:
from langchain.agents import ope

In [5]:
ORDER_DETAILS_EXTRACTION_LLAMA_PROMPT = """
Extract the following details from the provided Purchase Order Content:

1. Purchase Order Number/ID
2. Product details including:
   - Product Name
   - Quantity
   - Price per Unit
   - Total Net Value
3. Total Net Order Value (Excluding Tax)
4. Contact Person details including:
   - Name
   - Telephone/Phone Number
   - Email
   - Address
5. Estimated Time of Delivery

Purchase Order Content:

{po_content}

Output Format:
Format everthing into JSON

```json
{{po_id:[Purchase Order Number],
contact_person_details:{{name:[Name],phone_number:[Telephone/Phone Number], email:[Email], address: [Address]}},
products:[{{name:[Product Name],quantity:[Quantity], price_per_unit:[Price Per Unit (without currency)], net_value:[Total Net Value]}}],
eta:[Estimated Time of Delivery YYYY-MM-DD]
}}
```

"""

In [6]:
po_content = """

"""

In [215]:
DETAILS_EXTRACTION_PROMPT = """
Give me details in the invoice dividing to sections.



Invoice:
{attachement_content}
"""

In [216]:
attachement_content = """
XYZ Corporation
PURCHASE ORDER
Address: 123 Main Street, Suite 100, Cityville, State, ZIP
Contact Number: +1 (555) 123-4567
Default Email Address: info@xyzcorp.com
Website URL: www.xyzcorp.com
VENDORCUSTOMER
NAMENAME
PrasadPuttaJohn Smith
COMPANY NAMECOMPANY NAME
P.O. NUMBERDATE
2378112/10/2023
VassarLabsRedline Auto Center
ADDRESSADDRESS
9 Mindspace Road, Hyderabad, Telangana 500084
16040 S. US 27
Lansing, Michigan 48906
PHONEPHONE+919769275434517-367-7010EMAIL ADDRESSEMAIL ADDRESSsupport@vassarlabs.comjohnsmith@redline.comSHIPPING TERMSSHIPPING METHODDELIVERY DATE
Freight on BoardAir & Landmm/dd/yyyy
Product Description
Code
Quantity
Unit Price
Amount
304-98632Brake Discs, Pads & Calipers4111.36445.44
501-35587Control Arm260.93121.86
886-19386Suspension Lift Kit2399.83799.66
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
Note:
Payment shall be 30 days upon receipt of the items above.
Subtotal ($)
Discount (%)
10
Sales Tax (%)
12
Other Cost ($)
Shipping & Handling ($)
Total Amount ($)
1,366.96
136.70
164.04
500.00
800.00
2,694.30
"""

In [225]:
details_extraction_chain = llm_chain(
    prompt_template=ORDER_DETAILS_EXTRACTION_LLAMA_PROMPT
)

In [226]:
response = details_extraction_chain.run({"po_content": attachement_content})



> Entering new LLMChain chain...
Prompt after formatting:
System: 
Extract the following details from the provided Purchase Order Content:

1. Purchase Order Number/ID
2. Product details including:
   - Product Name
   - Quantity
   - Price per Unit
   - Total Net Value
3. Total Net Order Value (Excluding Tax)
4. Contact Person details including:
   - Name
   - Telephone/Phone Number
   - Email
   - Address
5. Estimated Time of Delivery

Purchase Order Content:


XYZ Corporation
PURCHASE ORDER
Address: 123 Main Street, Suite 100, Cityville, State, ZIP
Contact Number: +1 (555) 123-4567
Default Email Address: info@xyzcorp.com
Website URL: www.xyzcorp.com
VENDORCUSTOMER
NAMENAME
PrasadPuttaJohn Smith
COMPANY NAMECOMPANY NAME
P.O. NUMBERDATE
2378112/10/2023
VassarLabsRedline Auto Center
ADDRESSADDRESS
9 Mindspace Road, Hyderabad, Telangana 500084
16040 S. US 27
Lansing, Michigan 48906
PHONEPHONE+919769275434517-367-7010EMAIL ADDRESSEMAIL ADDRESSsupport@vassarlabs.comjohnsmith@redline.com

In [227]:
print(response)

{
  "po_id": "2378112",
  "contact_person_details": {
    "name": "John Smith",
    "phone_number": "+919769275434",
    "email": "johnsmith@redline.com",
    "address": "16040 S. US 27, Lansing, Michigan 48906"
  },
  "products": [
    {
      "name": "Brake Discs, Pads & Calipers",
      "quantity": "411",
      "price_per_unit": "11.36",
      "net_value": "445.44"
    },
    {
      "name": "Control Arm",
      "quantity": "260",
      "price_per_unit": "9.31",
      "net_value": "121.86"
    },
    {
      "name": "Suspension Lift Kit",
      "quantity": "2399",
      "price_per_unit": "799.83",
      "net_value": "799.66"
    }
  ],
  "eta": "2023-12-10"
}


In [228]:
import requests

In [5]:
string = "My name is prem"

# print(list(string))

In [229]:
data = {
    "model": "llama2:7b-chat-fp16",
    "prompt": ORDER_DETAILS_EXTRACTION_LLAMA_PROMPT.format(
        po_content=attachement_content
    ),
    "stream": False,
}

In [230]:
response = requests.post(url="https://genai.vassarlabs.com/api/generate", json=data)

In [231]:
response_json = json.loads(response.text)

In [1]:
CUSTOMER_AND_GEOGRAPHY_IDENFICATION_PROMPT = {
    "SYSTEM": """You are tasked with extracting geographic and customer details from provided emails.
Your goal is to identify the country, state, and address of the contact mentioned in the email.
Extract the customer's name, which is the company requesting the service in the email.""",
    "CONTEXT": """Instructions for extracting details from the email:
Geography of the contact:
Extract the country, state, and address.
If the country is UNIDENTIFIED, try to find it using the address. If not possible, mark it as "UNIDENTIFIED."
Customer:
Extract the customer_name from the email_content. The customer is the company which requests the service in the mail.
""",
    "DISPLAY": """
Provide the output in the following JSON format:
```json
{{
  "Customer": "[Customer_name]",
  "locations": {{
    "state": "[State]",
    "country": "[Country]",
    "Address": "[Address]"
  }}
}}```
""",
    "REMEMBER": """
""",
}

In [128]:
headers = {
    "Content-Type": "application/json",
}

In [144]:
data = {"inputs": prompt, "parameters": {"max_new_tokens": 1000}}

In [145]:
response = requests.post(
    url="https://genai.vassarlabs.com/api/v1/inference/generate", json=data
)

In [146]:
response

<Response [200]>

In [147]:
response_json = json.loads(response.text)

In [148]:
print(response_json["generated_text"])

Here is the structured output of the invoice breaking down the different sections:

**Invoice Information:**

* Invoice Number: OD-2324-004
* Invoice Date: 12-01-2024
* Terms: Due on Receipt
* Due Date: 12-01-2024

**Bill To and Ship To Information:**

* Bill To: Odisha Space Applications Centre, Plot no.45/48, Unit 16, Jaydev Vihar, Bhubaneswar, Khordha
* Ship To: Odisha Space Applications Centre, Plot no.45/48, Unit 16, Jaydev Vihar, Bhubaneswar, Khordha

**Item and Description:**

* Services: Milestone-6 (f) – Maintenance of BLUIS Project (Part 6 of 6) for the period from 08-July-23 to 07-January-24
* HSN Code: 9983141.00
* Quantity: 4,27,500.00
* Rate per unit: 4,27,500.00
* Total Amount: 5,04,450.00

**Tax Information:**

* CGST: 9%
* SGST: 9%
* Total Amount: 5,04,450.00

**Payment Information:**

* Payment Made: 4,53,150.00
* Balance Due: 0.00

**Notes:**

* Thanks for your business.
* Amount: 38,475.00
* Credits Applied: -8,550.00
* Amount Withheld: -42,750.00
* Balance Due: 0.0

In [ ]:
LLAMA2_CHAT_PROMPT_TEMPLATE = "<s>[INST]<<SYS>>{system_message}<</SYS>>{user_message}[/INST]{model_answer}</s>[INST]{follow_up_user_message}[/INST]"
MISTRAL_CHAT_PROMPT_TEMPLATE = "<s>[INST]<<SYS>>{system_message}<</SYS>>{user_message}[/INST]{model_answer}</s>[INST]{follow_up_user_message}[/INST]"


GEMMA_CHAT_PROMPT_TEMPLATE_2 = """
<start_of_turn>user
{user_message}
<end_of_turn>
<start_of_turn>model
{AI_message}
<end_of_turn>
<start_of_turn>user
{follow_up_user_message}
<end_of_turn>
<start_of_turn>model
"""

In [41]:
from enum import Enum


class Action(Enum):
    AI_RESPONDED = "AI_RESPONDED"
    AI_ASSISTED = "AI_ASSISTED"
    CSR = "CSR"


action_flow_name = "AI_RESPONDED"
action = Action[action_flow_name]
print(action.value)  # Output: Action.AI_RESPONDED

AI_RESPONDED


In [11]:
INTENT_SENTIMENT_CLASSIFICATION_PROMPT = {
    "SYSTEM": """You are an advanced email service AI system designed to classify the intent of incoming emails and discern the emotional tone conveyed within the message. 
    Your task is to intelligently categorize the emails into different intents and sub-intents based on conversations in email thread.
""",
    "CONTEXT": """Intents:
1. PURCHASE_ORDER:
   - Description: Managing purchase orders, including creation, status checks, and cancellation.
   - Sub-Intents:
     - NEW_PO: Creating a new purchase order.
     - CHECK_PO_STATUS: Checking the status of a purchase order.
     - PO_CANCELLATION: Requesting cancellation of a purchase order.

2. SHIPMENT_STATUS:
   - Description: Checking the status of a shipment.
   - Sub-Intents:
     - CHECK_SHIPMENT_STATUS: Checking the status of a shipment.

3. WARRANTY:
   - Description: Handling warranty-related inquiries and claims.
   - Sub-Intents:
     - INITIATE_WARRANTY_CLAIM: Initiating a warranty claim for faulty equipment.

4. SERVICE_REQUESTS:
   - Description: Requesting various services, maintenance, or troubleshooting tasks.
   - Sub-Intents:
     - ROUTINE_MAINTENANCE: Scheduled maintenance tasks to ensure functionality.
     - TROUBLESHOOTING: Diagnosing and resolving technical issues.
     - INSTALLATION: Installing new items or replacing existing ones.
     - UPGRADES_MODIFICATIONS: Upgrading or modifying items to enhance performance.
     - RESOLVED: Classifying emails that indicate the resolution of a previous issue or request.

Emotional Tone Detection:

- Classify the emotional tone of the message, indicating whether the customer is expressing HAPPY, ANXIOUS, UNHAPPY, or SATISFIED.

- Please analyze the emotional tone of the provided message and determine the predominant sentiment expressed by the customer. Choose the most appropriate emotional state from the following options:

SATISFIED: The customer expresses contentment, fulfillment, or approval.
HAPPY: The customer expresses positivity, joy, or contentment.
NEUTRAL: The customer expresses neither positive nor negative emotions.
ANXIOUS: The customer expresses worry, nervousness, or apprehension.
ANGRY: The customer expresses strong displeasure or hostility.
EXPEDITE: The customer expresses urgency or a need for immediate action.

Select the emotional state that best reflects the overall sentiment conveyed in the message. If the emotional tone is mixed or unclear, choose the category that aligns most closely with the prevailing emotion.     
""",
    "DISPLAY": """Provide the output in the following JSON format:
```json
{{
  "intent": "[Main Intent Classified]",
  "sub_intent": "[Sub-Intent Classified, if applicable]",
  "tone": "[Emotional Tone Detected]"
}}
```
""",
    "REMEMBER": """
""",
}

CUSTOMER_AND_GEOGRAPHY_IDENFICATION_PROMPT = {
    "SYSTEM": """You are tasked with extracting geographic and customer details from provided emails.
Your goal is to identify the country, state, and address of the contact mentioned in the email.
Extract the customer's name, which is the company requesting the service in the email.""",
    "CONTEXT": """Instructions for extracting details from the email:
Geography of the contact:
Extract the country, state, and address.
If the country is UNIDENTIFIED, try to find it using the address. If not possible, mark it as "UNIDENTIFIED."
Customer:
Extract the customer_name from the email_content. The customer is the company which requests the service in the mail.
""",
    "DISPLAY": """Provide the output in the following JSON format:
```json
{{
  "customer": "[Customer_name]",
  "locations": {
    "state": "[State]",
    "country": "[Country]",
    "address": "[Address]"
  },
}}
```
""",
    "REMEMBER": """
""",
}

TROUBLESHOOTING_DETAILS_EXTRACTION = {
    "SYSTEM": """
Your task is to extract specific details from provided troubleshooting emails.
Details to extract include Issue Type, issue description, Product details, location details, contact person details, severity/priority of the issue, task/work order number (if applicable), issue status, Estimated Deadline and any additional relevant information.
""",
    "CONTEXT": """
Instructions for extracting details from the email:

Issue Type (alarm_malfunction, equipment_failure, miscellaneous)
Issue Description
Product Details (Name)
Location Details:
Store/Location Name
Address
City
State/Province
Postal Code
Country
Contact Person details:
Name
Telephone/Phone Number
Email
Severity/Priority of the Issue (low, medium, high)
Task/Work Order Number (if applicable)
Status of the Issue (e.g., Open, In Progress, Closed)
Estimated Deadline to solve the issue (ETA)

Any additional relevant information provided

Email Content:
""",
    "DISPLAY": """Provide the output in the following JSON format:
```json
{{
  "issue_description": "[Issue Description]",
  "location_details": {{
    "name": "[Store/Location Name]",
    "address": "[Address]",
    "city": "[City]",
    "state_province": "[State/Province]",
    "postal_code": "[Postal Code]",
    "country": "[Country]"
  }},
  "contact_person_details": {{
    "name": "[Name]",
    "phone_number": "[Telephone/Phone Number]",
    "email": "[Email]"
  }},
  "severity_priority": "[Severity/Priority]",
  "task_work_order_number": "[Task/Work Order Number]",
  "issue_status": "[Status of the Issue]",
  "issue_type": "[Issue Type]",
  "product_details": {{
    "name": "[Name]"
  }},
  "eta": "[Estimated Deadline to solve the issue]",
  "additional_information": "[Any additional relevant information]"
}}```
""",
    "REMEMBER": """
""",
}

prompt_dimensions = {
    "customer_tier": {"uuid": ""},
    "geography": {"country": {"uuid": "", "state": {"uuid": ""}}},
    "intent": {"uuid": "", "sub_intent": {"uuid": ""}},
    "sentiment": {"uuid": ""},
    "comment": "",
    "action": {"uuid": ""},
}


TROUBLESHOOTING_RESPONSE_GENERATION = {
    "SYSTEM": """As a customer service representative, your role is to provide a personalized and helpful response to the troubleshooting email. Begin by summarizing the issue concisely in two lines, ensuring clarity for the customer service team. Then, craft an email response that acknowledges the reported issue, empathizes with the customer, offers solutions or assistance, and assures prompt resolution.""",
    "CONTEXT": """
Issue Reported: {question}
Responding to Customer Details: {customer_data}
""",
    "DISPLAY": """
Provide the output in the following JSON format:

```json
{{
  "summary": "[SUMMARY]",
  "response": "[EMAIL_RESPONSE]"
}}
```
""",
    "REMEMBER": """Craft the email response as if you were personally addressing the customer's concerns. Be empathetic, provide clear solutions or assistance, and assure the customer of prompt resolution. End with "Best Regards" for a professional closing.""",
}

In [15]:
ROUTINEMAINTENANCE_DETAILS_EXTRACTION = {
    "SYSTEM": """Your task is to extract specific details from emails requesting routine maintenance in service requests. 
Details to extract include the type of maintenance requested, equipment details, location details, contact person details, preferred maintenance schedule, task/work order number (if applicable), status of the request, estimated completion time, and any additional relevant information.
""",
    "CONTEXT": """
Instructions for extracting details from the email:

Maintenance Type (e.g., preventive, predictive, scheduled)
Equipment Details:
Equipment Name/Type
Equipment Model/Serial Number
Location Details:
Facility/Building Name
Floor/Room Number
Address
City
State/Province
Postal Code
Country
Contact Person Details:
Name
Telephone/Phone Number
Email
Preferred Maintenance Schedule
Task/Work Order Number (if applicable)
Status of the Request (e.g., Open, In Progress, Completed)
Estimated Completion Time
Additional Relevant Information Provided

Email Content:
""",
    "DISPLAY": """Provide the output in the following JSON format:
```json
{{
  "maintenance_type": "[Maintenance Type]",
  "equipment_details": {{
    "name": "[Equipment Name/Type]",
    "model_serial_number": "[Equipment Model/Serial Number]"
  }},
  "location_details": {{
    "facility_name": "[Facility/Building Name]",
    "floor_room_number": "[Floor/Room Number]",
    "address": "[Address]",
    "city": "[City]",
    "state_province": "[State/Province]",
    "postal_code": "[Postal Code]",
    "country": "[Country]"
  }},
  "contact_person_details": {{
    "name": "[Name]",
    "phone_number": "[Telephone/Phone Number]",
    "email": "[Email]"
  }},
  "preferred_maintenance_schedule": "[Preferred Maintenance Schedule]",
  "task_work_order_number": "[Task/Work Order Number]",
  "request_status": "[Status of the Request]",
  "estimated_completion_time": "[Estimated Completion Time]",
  "additional_information": "[Additional Relevant Information]"
}}
```
""",
    "REMEMBER": """
""",
}


ROUTINEMAINTENANCE_RESPONSE_GENERATION = {
    "SYSTEM": """As a customer service representative, your task is to provide a thoughtful and informative response to the routine maintenance request email. Begin by summarizing the maintenance request concisely in two lines, ensuring clarity for the customer service team. Then, craft an email response that acknowledges the maintenance request, confirms receipt of the request, provides details on the scheduled maintenance, and assures the customer of prompt service.""",
    "CONTEXT": """
Maintenance Requested: {question}
Customer Information: {customer_data}
""",
    "DISPLAY": """
Provide the output in the following JSON format:

```json
{{
  "summary": "[SUMMARY]",
  "response": "[EMAIL_RESPONSE]"
}}
```
""",
    "REMEMBER": """Craft the email response as if you were personally addressing the customer's maintenance request. Be courteous, provide clear information about the scheduled maintenance, and assure the customer of prompt service. End with "Best Regards" for a professional closing.""",
}

In [16]:
json_data = {
    "customer_tier": [{"uuid": "90d7fdc4-6fda-4eea-ae17-acbbb0255d92"}],
    "geography": [
        {
            "country": {
                "uuid": "d1d778b3-d9d6-4ef3-be92-afcce4c68063",
                "state": [{"uuid": "0fb8ade8-89bd-44a5-8bca-7366db429bf2"}],
            }
        }
    ],
    "intent": [
        {
            "uuid": "450e4fef-359a-456a-8218-3f8ca617e1f5",
            "sub_intent": [{"uuid": "1403f488-4458-4a82-8860-6fcd55a059b1"}],
        }
    ],
    "sentiment": "*",
}

In [1]:
PURCHASE_ORDER_SUMMARY_RESPONSE = {
    "SYSTEM": """You are a customer service representative for a company specializing in purchase orders. Your task is to generate a concise 2-line SUMMARY for the customer service team to quickly understand the purchase order email content. Moreover, craft a detailed and personalized EMAIL RESPONSE either confirming the purchase order if no issues are found or addressing any issues such as item unavailability, incorrect quantity, or pricing errors, and requesting a revised purchase order if necessary. Ensure the response feels personable and adaptive to the customer's tone.""",
    "CONTEXT": """Purchase Order Details: {question}

Responding to Customer : {customer_data}
""",
    "DISPLAY": """
Provide the output in the following JSON format:

```json
{{
  "summary": "[SUMMARY]",
  "response": "[EMAIL_RESPONSE]"
}}
```
""",
    "REMEMBER": """
Just provide the email body response. Do not provide any example details. End with Best Regards, and do not provide any placeholders.
""",
}

In [ ]:
INTENT_SENTIMENT_CLASSIFICATION_PROMPT = {
    "SYSTEM": """You are an advanced email service AI system designed to classify the intent of incoming emails and discern the emotional tone conveyed within the message. 
    Your task is to intelligently categorize the emails into different intents and sub-intents based on conversations in email thread.
""",
    "CONTEXT": """Intents:
1. PURCHASE_ORDER:
   - Description: Managing purchase orders, including creation, status checks, and cancellation.
   - Sub-Intents:
     - NEW_PO: Creating a new purchase order.
     - CHECK_PO_STATUS: Checking the status of a purchase order.
     - PO_CANCELLATION: Requesting cancellation of a purchase order.

2. SHIPMENT_STATUS:
   - Description: Checking the status of a shipment.
   - Sub-Intents:
     - CHECK_SHIPMENT_STATUS: Checking the status of a shipment.

3. WARRANTY:
   - Description: Handling warranty-related inquiries and claims.
   - Sub-Intents:
     - INITIATE_WARRANTY_CLAIM: Initiating a warranty claim for faulty equipment.

4. SERVICE_REQUEST:
   - Description: Requesting various services, maintenance, or troubleshooting tasks.
   - Sub-Intents:
     - ROUTINE_MAINTENANCE: Scheduled maintenance tasks to ensure functionality.
     - TROUBLESHOOTING: Diagnosing and resolving technical issues.
     - INSTALLATION: Installing new items or replacing existing ones.
     - UPGRADES_MODIFICATIONS: Upgrading or modifying items to enhance performance.
     - RESOLVED: Classifying emails that indicate the resolution of a previous issue or request.

Emotional Tone Detection:

- Classify the emotional tone of the message, indicating whether the customer is expressing HAPPY, NEUTRAL, ANXIOUS, ANGRY or EXPEDITE.

- Please analyze the emotional tone of the provided message and determine the predominant sentiment expressed by the customer. Choose the most appropriate emotional state from the following options:

HAPPY: The customer expresses positivity, joy, or contentment.
NEUTRAL: The customer expresses neither positive nor negative emotions.
ANXIOUS: The customer expresses worry, nervousness, or apprehension.
ANGRY: The customer expresses strong displeasure or hostility.
EXPEDITE: The customer expresses urgency or a need for immediate action.

Select the emotional state that best reflects the overall sentiment conveyed in the message. If the emotional tone is mixed or unclear, choose the category that aligns most closely with the prevailing emotion.     
""",
    "DISPLAY": """Provide the output in the following JSON format:
```json
{{
  "intent": "[Main Intent Classified]",
  "sub_intent": "[Sub-Intent Classified, if applicable]",
  "tone": "[Emotional Tone Detected]"
}}
```
""",
    "REMEMBER": """
""",
}

In [5]:
string = json.dumps(INTENT_SENTIMENT_CLASSIFICATION_PROMPT)

In [6]:
print(string)

{"SYSTEM": "You are a customer service representative for a company specializing in purchase orders. Your task is to generate a concise 2-line SUMMARY for the customer service team to quickly understand the purchase order email content. Moreover, craft a detailed and personalized EMAIL RESPONSE either confirming the purchase order if no issues are found or addressing any issues such as item unavailability, incorrect quantity, or pricing errors, and requesting a revised purchase order if necessary. Ensure the response feels personable and adaptive to the customer's tone.", "CONTEXT": "Purchase Order Details: {question}\n\nResponding to Customer : {customer_data}\n", "DISPLAY": "\nProvide the output in the following JSON format:\n\n```json\n{{\n  \"summary\": \"[SUMMARY]\",\n  \"response\": \"[EMAIL_RESPONSE]\"\n}}\n```\n", "REMEMBER": "\nJust provide the email body response. Do not provide any example details. End with Best Regards, and do not provide any placeholders.\n"}


In [3]:
from enum import Enum


class DimensionTypeNames(Enum):
    GEOGRAPHY_STATE = ("GEOGRAPHY_STATE",)
    GEOGRAPHY_COUNTRY = "GEOGRAPHY_COUNTRY"

In [17]:
print(DimensionTypeNames["GEOGRAPHY_STATE"])

DimensionTypeNames.GEOGRAPHY_STATE


In [34]:
string = {
    "SYSTEM": "\nYour task is to extract specific details from provided troubleshooting emails.\nDetails to extract include Issue Type, issue description, Product details, location details, contact person details, severity/priority of the issue, task/work order number (if applicable), issue status, Estimated Deadline and any additional relevant information.\n",
    "CONTEXT": "\nInstructions for extracting details from the email:\n\nIssue Type (alarm_malfunction, equipment_failure, miscellaneous)\nIssue Description\nProduct Details (Name)\nLocation Details:\nStore/Location Name\nAddress\nCity\nState/Province\nPostal Code\nCountry\nContact Person details:\nName\nTelephone/Phone Number\nEmail\nSeverity/Priority of the Issue (low, medium, high)\nTask/Work Order Number (if applicable)\nStatus of the Issue (e.g., Open, In Progress, Closed)\nEstimated Deadline to solve the issue (ETA)\n\nAny additional relevant information provided\n\nEmail Content:\n",
    "DISPLAY": 'Provide the output in the following JSON format:\n```json\n{{\n  "issue_description": "[Issue Description]",\n  "location_details": {{\n    "name": "[Store/Location Name]",\n    "address": "[Address]",\n    "city": "[City]",\n    "state_province": "[State/Province]",\n    "postal_code": "[Postal Code]",\n    "country": "[Country]"\n  }},\n  "contact_person_details": {{\n    "name": "[Name]",\n    "phone_number": "[Telephone/Phone Number]",\n    "email": "[Email]"\n  }},\n  "severity_priority": "[Severity/Priority]",\n  "task_work_order_number": "[Task/Work Order Number]",\n  "issue_status": "[Status of the Issue]",\n  "issue_type": "[Issue Type]",\n  "product_details": {\n    "name": "[Name]"\n  },\n  "eta": "[Estimated Deadline to solve the issue]",\n  "additional_information": "[Any additional relevant information]"\n}}```\n',
    "REMEMBER": "\n",
}

In [37]:
pprint(string)

{'CONTEXT': '\n'
            'Instructions for extracting details from the email:\n'
            '\n'
            'Issue Type (alarm_malfunction, equipment_failure, miscellaneous)\n'
            'Issue Description\n'
            'Product Details (Name)\n'
            'Location Details:\n'
            'Store/Location Name\n'
            'Address\n'
            'City\n'
            'State/Province\n'
            'Postal Code\n'
            'Country\n'
            'Contact Person details:\n'
            'Name\n'
            'Telephone/Phone Number\n'
            'Email\n'
            'Severity/Priority of the Issue (low, medium, high)\n'
            'Task/Work Order Number (if applicable)\n'
            'Status of the Issue (e.g., Open, In Progress, Closed)\n'
            'Estimated Deadline to solve the issue (ETA)\n'
            '\n'
            'Any additional relevant information provided\n'
            '\n'
            'Email Content:\n',
 'DISPLAY': 'Provide the output in the follow

In [1]:
{
    "customer_tier": [{"uuid": "90d7fdc4-6fda-4eea-ae17-acbbb0255d92"}],
    "geography": {
        "country": [
            {
                "uuid": "d1d778b3-d9d6-4ef3-be92-afcce4c68063",
                "state": [
                    {
                        "uuid": "0fb8ade8-89bd-44a5-8bca-7366db429bf2",
                    },
                ],
            },
        ]
    },
    "intent": [
        {
            "uuid": "",
            "sub_intent": [
                {"uuid": ""},
            ],
        },
    ],
    "sentiment": [{"uuid": ""}],
}

{'customer_tier': [{'uuid': '90d7fdc4-6fda-4eea-ae17-acbbb0255d92'}],
 'geography': {'country': [{'uuid': 'd1d778b3-d9d6-4ef3-be92-afcce4c68063',
    'state': [{'uuid': '0fb8ade8-89bd-44a5-8bca-7366db429bf2'}]}]},
 'intent': [{'uuid': '', 'sub_intent': [{'uuid': ''}]}],
 'sentiment': [{'uuid': ''}]}

In [2]:
from datetime import datetime

print(datetime.now().timestamp())

1713857878.621411


In [ ]:
json_data = {
    "purchaseOrderInfo": {
        "purchaseOrderNumber": "PO12345",
        "orderDate": "2023-05-10",
        "eta": "2023-06-10",
        "currency": "USD",
    },
    "buyerInformation": {
        "buyerName": "Acme Corporation",
        "buyerAddress": "123 Elm Street, Springfield, IL 62701, USA",
        "buyerContact": {"phone": "+1-234-567-8900", "email": "contact@acme.com"},
    },
    "sellerInformation": {
        "sellerName": "Widgets Inc.",
        "sellerAddress": "456 Oak Avenue, Metropolis, NY 10001, USA",
        "sellerContact": {"phone": "+1-987-654-3210", "email": "sales@widgets.com"},
    },
    "orderDetails": {
        "items": [
            {
                "itemDescription": "Widget A",
                "itemCode": "WA123",
                "quantity": 10,
                "unitPrice": 15.00,
                "totalPrice": 150.00,
            },
            {
                "itemDescription": "Widget B",
                "itemCode": "WB456",
                "quantity": 5,
                "unitPrice": 25.00,
                "totalPrice": 125.00,
            },
        ],
        "subTotal": 275.00,
        "tax": 22.00,
        "totalAmount": 299.00,
    },
    "shippingInformation": {
        "shippingAddress": "123 Elm Street, Springfield, IL 62701, USA",
        "shippingInstructions": "Leave at the front desk.",
    },
    "additionalInformation": {"comments": "Please expedite the shipment."},
}

In [10]:
json_data = json.loads(
    '{"purchaseorderinformation": {"purchaseOrderNumber": "73414", "orderDate": "4/11/2024", "eta": "4/12/2024", "currency": "USD"}, "buyerinformation": {"buyerName": "Axel Reyes", "buyerAddress": "Spencer Technologies, Inc, 10 Trotter Drive, Medway, MA 02053, US", "buyerContact": {"phone": "", "email": "APinvoice@spencertech.com"}}, "sellerinformation": {"sellerName": "Sensormatic", "sellerAddress": "6600 Congress Ave, ATTN: William Burkholder, Boca Raton, FL 33487, US", "sellerContact": {"phone": "", "email": ""}}, "orderdetails": {"items": [{"itemDescription": "IA-POE-90-U00 - PoE Injector, 90W, Universal", "itemCode": "", "quantity": "2", "unitPrice": "$150.00", "totalPrice": "$300.00"}, {"itemDescription": "", "itemCode": "", "quantity": "", "unitPrice": "", "totalPrice": ""}], "subTotal": "", "tax": "", "totalAmount": "$300.00"}, "shippinginformation": {"shippingAddress": "Academy Sports & Outdoors-69, 445 Forest Square St, Longview, TX 75605, US", "shippingInstructions": ""}, "additionalinformation": {"comments": "Please confirm price and del to procurement@spencertech.com"}}'
)

In [11]:
json_data

{'purchaseorderinformation': {'purchaseOrderNumber': '73414',
  'orderDate': '4/11/2024',
  'eta': '4/12/2024',
  'currency': 'USD'},
 'buyerinformation': {'buyerName': 'Axel Reyes',
  'buyerAddress': 'Spencer Technologies, Inc, 10 Trotter Drive, Medway, MA 02053, US',
  'buyerContact': {'phone': '', 'email': 'APinvoice@spencertech.com'}},
 'sellerinformation': {'sellerName': 'Sensormatic',
  'sellerAddress': '6600 Congress Ave, ATTN: William Burkholder, Boca Raton, FL 33487, US',
  'sellerContact': {'phone': '', 'email': ''}},
 'orderdetails': {'items': [{'itemDescription': 'IA-POE-90-U00 - PoE Injector, 90W, Universal',
    'itemCode': '',
    'quantity': '2',
    'unitPrice': '$150.00',
    'totalPrice': '$300.00'},
   {'itemDescription': '',
    'itemCode': '',
    'quantity': '',
    'unitPrice': '',
    'totalPrice': ''}],
  'subTotal': '',
  'tax': '',
  'totalAmount': '$300.00'},
 'shippinginformation': {'shippingAddress': 'Academy Sports & Outdoors-69, 445 Forest Square St, Lo

In [ ]:
{
    "purchaseorderinformation": {
        "purchaseOrderNumber": "73414",
        "orderDate": "4/11/2024",
        "eta": "4/12/2024",
        "currency": "USD",
    },
    "buyerinformation": {
        "buyerName": "Axel Reyes",
        "buyerAddress": "Spencer Technologies, Inc, 10 Trotter Drive, Medway, MA 02053, US",
        "buyerContact": {"phone": "", "email": "APinvoice@spencertech.com"},
    },
    "sellerinformation": {
        "sellerName": "Sensormatic",
        "sellerAddress": "6600 Congress Ave, ATTN: William Burkholder, Boca Raton, FL 33487, US",
        "sellerContact": {"phone": "", "email": ""},
    },
    "orderdetails": {
        "items": [
            {
                "itemDescription": "Sensormatic SlimPad Deactivator, Universal Controller, Remote Alarm Module, Power Cord",
                "itemCode": 'ZBSMP-"{"purchaseorderinformation": {"purchaseOrderNumber": "73414", "orderDate": "4/11/2024", "eta": "4/12/2024", "currency": "USD"}, "buyerinformation": {"buyerName": "Axel Reyes", "buyerAddress": "Spencer Technologies, Inc, 10 Trotter Drive, Medway, MA 02053, US", "buyerContact": {"phone": "", "email": "APinvoice@spencertech.com"}}, "sellerinformation": {"sellerName": "Sensormatic", "sellerAddress": "6600 Congress Ave, ATTN: William Burkholder, Boca Raton, FL 33487, US", "sellerContact": {"phone": "", "email": ""}}, "orderdetails": {"items": [{"itemDescription": "ZBSMP-SL - Sensormatic SlimPad Deactivator, Universal Controller, Remote Alarm Module, Power Cord", "itemCode": "", "quantity": "2", "unitPrice": "$707.25", "totalPrice": "$1414.50"}, {"itemDescription": "LABOR - LABOR", "itemCode": "", "quantity": "1", "unitPrice": "$601.54", "totalPrice": "$601.54"}], "subTotal": "$2016.04", "tax": "", "totalAmount": "$2016.04"}, "shippinginformation": {"shippingAddress": "Academy Sports & Outdoors-69, 445 Forest Square St, Longview, TX 75605, US", "shippingInstructions": ""}, "additionalinformation": {"comments": "Please confirm price and del to procurement@spencertech.com"}}")SL - ',
                "quantity": "2",
                "unitPrice": "$707.25",
                "totalPrice": "$1414.50",
            },
            {
                "itemDescription": "LABOR",
                "itemCode": "LABOR",
                "quantity": "1",
                "unitPrice": "$601.54",
                "totalPrice": "$601.54",
            },
        ],
        "subTotal": "$2016.04",
        "tax": "",
        "totalAmount": "$2016.04",
    },
    "shippinginformation": {
        "shippingAddress": "Academy Sports & Outdoors-69, 445 Forest Square St, Longview, TX 75605, US",
        "shippingInstructions": "",
    },
    "additionalinformation": {
        "comments": "Please confirm price and del to procurement@spencertech.com"
    },
}

In [8]:
string = json.dumps(
    {
        "purchaseorderinformation": {
            "purchaseOrderNumber": "73414",
            "orderDate": "4/11/2024",
            "eta": "4/12/2024",
            "currency": "USD",
        },
        "buyerinformation": {
            "buyerName": "Axel Reyes",
            "buyerAddress": "Spencer Technologies, Inc, 10 Trotter Drive, Medway, MA 02053, US",
            "buyerContact": {"phone": "", "email": "APinvoice@spencertech.com"},
        },
        "sellerinformation": {
            "sellerName": "Sensormatic",
            "sellerAddress": "6600 Congress Ave, ATTN: William Burkholder, Boca Raton, FL 33487, US",
            "sellerContact": {"phone": "", "email": ""},
        },
        "orderdetails": {
            "items": [
                {
                    "itemDescription": "Sensormatic SlimPad Deactivator, Universal Controller, Remote Alarm Module, Power Cord",
                    "itemCode": "ZBSMP-SL - ",
                    "quantity": "2",
                    "unitPrice": "$707.25",
                    "totalPrice": "$1414.50",
                },
                {
                    "itemDescription": "LABOR",
                    "itemCode": "LABOR",
                    "quantity": "1",
                    "unitPrice": "$601.54",
                    "totalPrice": "$601.54",
                },
            ],
            "subTotal": "$2016.04",
            "tax": "",
            "totalAmount": "$2016.04",
        },
        "shippinginformation": {
            "shippingAddress": "Academy Sports & Outdoors-69, 445 Forest Square St, Longview, TX 75605, US",
            "shippingInstructions": "",
        },
        "additionalinformation": {
            "comments": "Please confirm price and del to procurement@spencertech.com"
        },
    }
)

In [9]:
print(string)

{"purchaseorderinformation": {"purchaseOrderNumber": "73414", "orderDate": "4/11/2024", "eta": "4/12/2024", "currency": "USD"}, "buyerinformation": {"buyerName": "Axel Reyes", "buyerAddress": "Spencer Technologies, Inc, 10 Trotter Drive, Medway, MA 02053, US", "buyerContact": {"phone": "", "email": "APinvoice@spencertech.com"}}, "sellerinformation": {"sellerName": "Sensormatic", "sellerAddress": "6600 Congress Ave, ATTN: William Burkholder, Boca Raton, FL 33487, US", "sellerContact": {"phone": "", "email": ""}}, "orderdetails": {"items": [{"itemDescription": "Sensormatic SlimPad Deactivator, Universal Controller, Remote Alarm Module, Power Cord", "itemCode": "ZBSMP-SL - ", "quantity": "2", "unitPrice": "$707.25", "totalPrice": "$1414.50"}, {"itemDescription": "LABOR", "itemCode": "LABOR", "quantity": "1", "unitPrice": "$601.54", "totalPrice": "$601.54"}], "subTotal": "$2016.04", "tax": "", "totalAmount": "$2016.04"}, "shippinginformation": {"shippingAddress": "Academy Sports & Outdoors

In [4]:
json_data = {
    "SYSTEM": "You are a customer service representative for a company specializing in purchase orders. Your task is to generate a concise 2-line SUMMARY for the customer service team to quickly understand the purchase order email content. Moreover, craft a detailed and personalized EMAIL RESPONSE either confirming the purchase order if no issues are found or addressing any issues such as item unavailability, incorrect quantity, or pricing errors, and requesting a revised purchase order if necessary. Ensure the response feels personable and adaptive to the customers tone.",
    "CONTEXT": '\nPO Details: {question}\n\nEmail Response Template:\n1. Start with "Dear Customer,\nI hope this email finds you well. Thank you for purchase order "\n2. Either confirming the purchase order with order details if no issues are found or addressing any issues such as item unavailability, incorrect quantity, or pricing errors, and requesting a revised purchase order if necessary\n3. End with in new line"If you have any further questions or concerns, please do not hesitate to contact us.\n\n\nBest regards,"\n\n',
    "DISPLAY": '\nProvide the output in the following JSON format:\n\n```json\n{{\n  "summary": "[SUMMARY]",\n  "response": "[EMAIL_RESPONSE]"\n}}\n```\n',
    "REMEMBER": "\nJust provide the email body response in visually appealing with new line characters, spaces. Do not provide any example details. End with Best Regards, and do not provide any placeholders. \n",
}

In [7]:
string = json.dumps(json_data)
print(string)

{"SYSTEM": "You are a customer service representative for a company specializing in purchase orders. Your task is to generate a concise 2-line SUMMARY for the customer service team to quickly understand the purchase order email content. Moreover, craft a detailed and personalized EMAIL RESPONSE either confirming the purchase order if no issues are found or addressing any issues such as item unavailability, incorrect quantity, or pricing errors, and requesting a revised purchase order if necessary. Ensure the response feels personable and adaptive to the customers tone.", "CONTEXT": "\nPO Details: {question}\n\nEmail Response Template:\n1. Start with \"Dear Customer,\nI hope this email finds you well. Thank you for purchase order \"\n2. Either confirming the purchase order with order details if no issues are found or addressing any issues such as item unavailability, incorrect quantity, or pricing errors, and requesting a revised purchase order if necessary\n3. End with in new line\"If 

In [11]:
string = 
json_data = json.loads(string)
print(json_data)

JSONDecodeError: Invalid control character at: line 1 column 591 (char 590)

In [25]:
CHECK_PO_STATUS_SUMMARY_RESPONSE = {
    "SYSTEM": """As a customer service representative, your task is to generate a concise 2-line SUMMARY and a detailed EMAIL RESPONSE regarding the status of a purchase order.

Order Status details:
{question}

Email Response template:

1. Start with "Dear Customer,
I hope this email finds you well. Thank you for reaching out regarding the status of your order."
2. Provide relevant details about the status of the order.
3. End with "If you have any further questions or concerns, please do not hesitate to contact us.

Best regards,
Customer Service Representative"
""",
    "DISPLAY": """
Provide the output in the following JSON format:

```json
{{
  "summary": "[SUMMARY]",
  "response": "[EMAIL_RESPONSE]"
}}
```
""",
    "REMEMBER": """
Ensure the email response includes specific details related to the order status. Also, email response should be in visually appealing with new line characters, spaces. Omit any placeholders and conclude with "Best Regards."
""",
}
CHECK_SHIPMENT_STATUS_SUMMARY_RESPONSE ={
  "SYSTEM": """As a customer service representative, your task is to generate a concise 2-line SUMMARY and a detailed EMAIL RESPONSE regarding the shipment status of the order.
    
Shipment Status details:
{question}

Email Response template:

1. Start with "Dear Customer, 
I hope this email finds you well. Thank you for reaching out regarding the status of your order."
2. Provide relevant details about the shipment status of the order.
3. End with "If you have any further questions or concerns, please do not hesitate to contact us.

Best regards,
Customer Service Representative"
""",
  "DISPLAY": """
Provide the output in the following JSON format:

```json
{{
  "summary": "[SUMMARY]",
  "response": "[EMAIL_RESPONSE]"
}}
```
""",
  "REMEMBER": """
Ensure the email response includes specific details related to the orders shipment status. Also, email response should be in visually appealing with new line characters, spaces. Omit any placeholders and conclude with "Best Regards."
"""
}
PURCHASE_ORDER_SUMMARY_RESPONSE ={
  "SYSTEM": """You are a customer service representative for a company specializing in purchase orders. Your task is to generate a concise 2-line SUMMARY for the customer service team to quickly understand the purchase order email content. Moreover, craft a detailed and personalized EMAIL RESPONSE either confirming the purchase order if no issues are found or addressing any issues such as item unavailability, incorrect quantity, or pricing errors. Ensure the response feels personable and adaptive to the customers tone.""",
  "CONTEXT": """
PO Details: {question}

Email Response Template:
1. Start with "Dear Customer,

I hope this email finds you well. Thank you for your purchase order."
2. If no issues are found: Confirm the purchase order with the order number/id and add "\nWe will keep you updated on the status of your order."
   Else If items are not available or incorrect quantity: Respond with "\nUnfortunately, we are currently out of stock for the [item(s)] in your order and cannot process it at this time." Add "We apologize for any inconvenience this may cause."
   Else: Address any issues such as pricing errors.
3. Finally end with a new line "\nIf you have any further questions or concerns, please do not hesitate to contact us.

Best regards,
Customer Service Representative"
""",
  "DISPLAY": """
Provide the output in the following JSON format:

```json
{{
  "summary": "[SUMMARY]",
  "response": "[EMAIL_RESPONSE]"
}}
```
""",
  "REMEMBER": """
Just provide the email body response in visually appealing with new line characters, spaces. Do not provide any example details. End with Best Regards, and do not provide any placeholders.
"""
}


In [26]:
string = json.dumps(PURCHASE_ORDER_SUMMARY_RESPONSE)
print(string)

{"SYSTEM": "You are a customer service representative for a company specializing in purchase orders. Your task is to generate a concise 2-line SUMMARY for the customer service team to quickly understand the purchase order email content. Moreover, craft a detailed and personalized EMAIL RESPONSE either confirming the purchase order if no issues are found or addressing any issues such as item unavailability, incorrect quantity, or pricing errors. Ensure the response feels personable and adaptive to the customers tone.", "CONTEXT": "\nPO Details: {question}\n\nEmail Response Template:\n1. Start with \"Dear Customer,\n\nI hope this email finds you well. Thank you for your purchase order.\"\n2. If no issues are found: Confirm the purchase order with the order number/id and add \"\nWe will keep you updated on the status of your order.\"\n   Else If items are not available or incorrect quantity: Respond with \"\nUnfortunately, we are currently out of stock for the [item(s)] in your order and 

In [27]:
string = json.dumps(CHECK_PO_STATUS_SUMMARY_RESPONSE)
print(string)

{"SYSTEM": "As a customer service representative, your task is to generate a concise 2-line SUMMARY and a detailed EMAIL RESPONSE regarding the status of a purchase order.\n\nOrder Status details:\n{question}\n\nEmail Response template:\n\n1. Start with \"Dear Customer,\nI hope this email finds you well. Thank you for reaching out regarding the status of your order.\"\n2. Provide relevant details about the status of the order.\n3. End with \"If you have any further questions or concerns, please do not hesitate to contact us.\n\nBest regards,\nCustomer Service Representative\"\n", "DISPLAY": "\nProvide the output in the following JSON format:\n\n```json\n{{\n  \"summary\": \"[SUMMARY]\",\n  \"response\": \"[EMAIL_RESPONSE]\"\n}}\n```\n", "REMEMBER": "\nEnsure the email response includes specific details related to the order status. Also, email response should be in visually appealing with new line characters, spaces. Omit any placeholders and conclude with \"Best Regards.\"\n"}


In [28]:
string = json.dumps(CHECK_SHIPMENT_STATUS_SUMMARY_RESPONSE)
print(string)

{"SYSTEM": "As a customer service representative, your task is to generate a concise 2-line SUMMARY and a detailed EMAIL RESPONSE regarding the shipment status of the order.\n    \nShipment Status details:\n{question}\n\nEmail Response template:\n\n1. Start with \"Dear Customer, \nI hope this email finds you well. Thank you for reaching out regarding the status of your order.\"\n2. Provide relevant details about the shipment status of the order.\n3. End with \"If you have any further questions or concerns, please do not hesitate to contact us.\n\nBest regards,\nCustomer Service Representative\"\n", "DISPLAY": "\nProvide the output in the following JSON format:\n\n```json\n{{\n  \"summary\": \"[SUMMARY]\",\n  \"response\": \"[EMAIL_RESPONSE]\"\n}}\n```\n", "REMEMBER": "\nEnsure the email response includes specific details related to the orders shipment status. Also, email response should be in visually appealing with new line characters, spaces. Omit any placeholders and conclude with \

In [5]:
import json
json_data = {
    "code": {
        "module": "EmailApp.Services.email_classification_service",
        "type": "class",
        "class": {
            "name": "EmailClassificationService",
            "constructor_params": {
                "customer_uuid" :  None,
                "conversation_uuid" : None
            },
            "method": {
                "name": "initialize_flow",
                "params": {
                }
            }
        }
    },
    "next_step_uuid": "45d3ff49-3c4b-416f-8dc7-69819b9da6d7"
}

{
    "code": {
        "module": "EmailApp.Services.details_extraction_service",
        "type": "class",
        "class": {
            "name": "DetailsExtractionService",
            "constructor_params": {
                "customer_uuid" :  None,
                "conversation_uuid" : None
            },
            "method": {
                "name": "extract_details",
                "params": {
                }
            }
        }
    }
}
string = json.dumps(json_data)

In [6]:
string

'{"code": {"module": "EmailApp.Services.details_extraction_service", "type": "class", "class": {"name": "DetailsExtractionService", "constructor_params": {"customer_uuid": null, "conversation_uuid": null}, "method": {"name": "extract_details", "params": {}}}}}'